Choice VD-VAE

- Hyperparameter tuning: 
> - Complexity of Architecture [Default, Reasoned custom one, One VD-VAE from Child 2021] 
> - Window Size [32, 64, 128]
> - Feature encoding RP, MTF, GADF, GASF  [GASF] 
> - Optimizer [DEFAULT] 
> - Loss Function [DEFAULT]
> - n_epochs=3 [50]
> - batch_size=256 [128, 64, 32]



Action plan:
- Find model with best setting on AMZN dataset 
- Iterate through all data sets

- Hyperparameter round 1: 
> - Complexity of Architecture [Default, Reasoned custom one, One VD-VAE from Child 2021] 
> - Window Size [64, 128, 256]
> - Feature encoding RP, MTF, GADF, GASF  [GASF] 
> - Optimizer [DEFAULT] 
> - Loss Function [DEFAULT]
> - n_epochs=5 [5]
> - batch_size=256 [256]
> - only on AMZN


- Hyperparameter round 2: 
> - Complexity of Architecture [Default, Reasoned custom one, One VD-VAE from Child 2021] 
> - Window Size [128]
> - Feature encoding RP, MTF, GADF, GASF  [GASF] 
> - Optimizer [DEFAULT] 
> - Loss Function [DEFAULT]
> - n_epochs=3 [50]
> - batch_size=256 [64]

Goal ==> build complexity of model architecture 


# VD-VAE

# Section 0 - Set-up 

In [ ]:
pip install pyts

     |████████████████████████████████| 2.5MB 10.3MB/s 


In [ ]:
! pip install kornia
!git clone https://www.github.com/EugenHotaj/pytorch-generative pytorch_generative_all 
!mv pytorch_generative_all/pytorch_generative pytorch_generative

     |████████████████████████████████| 286kB 11.4MB/s 
Cloning into 'pytorch_generative_all'...
remote: Enumerating objects: 1183, done.
remote: Counting objects: 100% (428/428), done.
remote: Compressing objects: 100% (330/330), done.


In [ ]:
import torch
import torchvision
import torch.optim as optim
import argparse
import matplotlib
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torch.nn import functional as F
import tensorflow as tf
from tqdm import tqdm
matplotlib.style.use('ggplot')

# Get things ready
import pytorch_generative as pg
#import pytorch_generative.models.vd_vae as vd_vae

# import function 
import numpy as np 
from os.path import join
from os import getcwd
import pandas as pd

from google.colab import drive
import sys

#import functions
import pyts
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from pyts.image import GramianAngularField
from pyts.image import RecurrencePlot
from pyts.image import MarkovTransitionField

import numpy as np
from sklearn.model_selection import train_test_split

import gc

from sklearn.metrics import f1_score, confusion_matrix



In [ ]:
4/1AY0e-g59WMj9fEsVy1RgqitDvG4-Rorn1m4KQYdP6XFNIUvBwZAC_36hh-4
## set-up path
drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/Master in Data Science and Society/Master Thesis/Code & Data Set')
path = '/content/drive/My Drive/Master in Data Science and Society/Master Thesis/Code & Data Set/labels/realAdExchange/twitterADVec_exchange-4_cpm_results.csv'

## load data
df = pd.read_csv(path)
ts = np.array(df['value'])

# Section 1 - Pre-processing


## Functions

In [ ]:
# functions 
def preprocess_images(images):
  images = images.reshape((images.shape[0], 1, window_size,window_size))
  return images

def rolling_window(ts, window):
  """  
  Segreates times series to windows of size S 
  in rolling fashion 

  For example: 
  window0 = [x0, x1, x2]
  window1 = [x1, x2, x3]
  ...
  
  Arguments: 
  > ts = univariate time series 
  > window_size = int length of window
  """
  shape = ts.shape[:-1] + (ts.shape[-1] - window + 1, window)
  strides = ts.strides + (ts.strides[-1],)
  return np.lib.stride_tricks.as_strided(ts, shape=shape, strides=strides)
  
      # """References"""
      # [1]https://rigtorp.se/2011/01/01/rolling-statistics-numpy.html
      # [2]https://stackoverflow.com/questions/27852343/split-python-sequence-time-series-array-into-subsequences-with-overlap


## Dataset prep

In [ ]:
#Prep Dataset List 
dataset_loader_list = {}

#set batch size
batch_size = 64

# prep
for window_size in [64]: #, 128]: 

  #set up model
  gasf = GramianAngularField(image_size=window_size, method='summation')# , sample_range=(0,1))
  
  #GAF
  ts_windows = rolling_window(ts, window_size)
  
  
  ## set up list storing img of ts window
  ts_windows_img = []

  for ts_window in ts_windows:

    ### GASF Transform
    X_gasf = gasf.fit_transform(ts_window.reshape(1,-1))

    ts_windows_img.append(X_gasf[0])

  # convert to tensor
  tensor_img = torch.Tensor(ts_windows_img)

  # Data loader set up 
  X_train, X_val, _, _ = train_test_split(tensor_img, 
                                          tensor_img,
                                          test_size=0.1, 
                                          random_state=42)

  ## Dataset GAF
  data_loader_train_GAF = torch.utils.data.DataLoader(preprocess_images(X_train),
                                            batch_size=batch_size,
                                            shuffle=False)

  data_loader_val_GAF = torch.utils.data.DataLoader(preprocess_images(X_val),
                                            batch_size=batch_size ,
                                            shuffle=False)

  data_loader_test_GAF = torch.utils.data.DataLoader(preprocess_images(tensor_img),
                                            batch_size=batch_size ,
                                            shuffle=False)
  
  dataset_loader_list[f'GAF{str(window_size)}'] = (window_size,
                                                   data_loader_train_GAF,
                                                   data_loader_val_GAF,
                                                   data_loader_test_GAF)




# Train VAE 

In [ ]:
import pytorch_generative.models.vae.vd_vae as vd_vae
# Hyperparameter - unchanged for now

## Loss, Optimizer, sample 

def loss_fn(x, _, preds):
        preds, kl_div = preds
        recon_loss = F.mse_loss(preds, x, reduction="none")
        recon_loss = recon_loss.sum(dim=(1, 2, 3))
        elbo = recon_loss + kl_div
        log.append((recon_loss.mean(),kl_div.mean(),elbo.mean()))

        return {
            "recon_loss": recon_loss.mean(),
            "kl_div": kl_div.mean(),
            "loss": elbo.mean(),
        }

def sample_fn(model):
        sample = torch.sigmoid(model.sample(n_samples=16))
        return torch.where(
            sample < 0.5, torch.zeros_like(sample), torch.ones_like(sample)
        )


In [ ]:
eval_score = {}
#n_epochs=5 #seems to overfit after 5 epochs
log_dir="/tmp/run"
n_gpus=1
device_id=0
debug_loader=None
log=[]

#     ws, train, val, test = dataset_loader_list[f'{ds}'] 
ws, train, val, test = dataset_loader_list[f'GAF64'] 

#for ds_key in dataset_loader_list.keys():
    #print(f'Dataset:{ds_key}')
    #ws, train, val, test = dataset_loader_list[ds_key]

lc, hc, bc, stack, lr, n_epochs
for lc in [2,4]:
  
    for hc, bc in [(32,8),(64,16),(128,32)]: #[(32,8),(64,16),(128,32)]:
      
      for stack in [(1,1,1,1,1,1)], (7,7,7,7,7,4), (4,4,4,4,4,2)]:

        for lr in [0.00001]:

          for n_epochs in [5,10]:
            
          

            stack_configs = [
              vd_vae.StackConfig(n_encoder_blocks=stack[5], n_decoder_blocks=stack[5]),
              vd_vae.StackConfig(n_encoder_blocks=stack[4], n_decoder_blocks=stack[4]),
              vd_vae.StackConfig(n_encoder_blocks=stack[3], n_decoder_blocks=stack[3]),
              vd_vae.StackConfig(n_encoder_blocks=stack[2], n_decoder_blocks=stack[2]),
              vd_vae.StackConfig(n_encoder_blocks=stack[1], n_decoder_blocks=stack[1]),
              vd_vae.StackConfig(n_encoder_blocks=stack[0], n_decoder_blocks=stack[0]),
            ]

            print(stack_configs)

          
            model = vd_vae.VeryDeepVAE(
                  in_channels=1,
                  out_channels=1,
                  input_resolution=ws,
                  stack_configs=stack_configs,
                  latent_channels=lc, #4  #set low since time series data less informative
                  hidden_channels=hc, #16
                  bottleneck_channels=bc, #8
          )

              # loss 
            optimizer = optim.Adam(model.parameters(), lr=lr)

              #model trainer
            model_trainer = pg.trainer.Trainer(
                  model=model,
                  loss_fn=loss_fn,
                  optimizer=optimizer,
                  train_loader=train,
                  eval_loader=val,
                  sample_epochs=1,
                  sample_fn=sample_fn,
                  log_dir=log_dir,
                  n_gpus=n_gpus,
                  device_id=device_id,
                )
              # save/ load model 
            model_trainer.interleaved_train_and_eval(n_epochs, restore=False)

            #torch.save(model, f'/content/drive/My Drive/Master in Data Science and Society/Master Thesis/Code & Data Set/trained_models/VD_VAE_AMZN_GAF{str(ws)}_{str(sum(stack*2))}_{str((lc, hc, bc))}.h5')
            #model = torch.load(f'/content/drive/My Drive/Master in Data Science and Society/Master Thesis/Code & Data Set/trained_models/VD_VAE_AMZN_GAF{str(ws)}_{str(sum(stack*2))}_{str(latent_channels)}.h5')

              # evaluate 
            model_eval = model.train(False)

            MSEs_of_inputs = []
            for b in test: 
                        gc.collect()
                        x = model_eval(b.cuda())
                        x = x[0].detach()

                        #MSE of inputs of batch
                        sq_diff = torch.square(x - b.cuda())
                        Batch_MSE = 0.5*torch.sum(sq_diff, [1,2,3])

                        MSEs_of_inputs.append(Batch_MSE)
                        gc.collect()

              # convert to tensor 
            MSEs_of_inputs = torch.cat(MSEs_of_inputs)

              # Threshold 
            std, mean = torch.std_mean(MSEs_of_inputs) # returns standard-deviation and mean

            y_pred = []
            for ind, r_error in enumerate(MSEs_of_inputs): 
                
                # Anomaly detection
                if r_error > (mean + 2*std) or r_error < (mean - 2*std):
                      
                    #print('Anomaly')
                    #df['pred'][ind+(w-1)] = 1 #w is width set initially -1 due to Xue et al implementation
                    y_pred.append(1)

                else: 
                    #print('Normal')
                    #df['pred'][ind+(w-1)] = 0
                    y_pred.append(0)

            y_true = df['label'][int((0.5*ws)-1):int(-0.5*ws)].tolist()

            eval_score[f'VD_VAE_GAF{(lc, hc, bc, stack, lr, n_epochs)}'] = ( f1_score(y_true, y_pred),confusion_matrix(y_true, y_pred)) #assoiate with settings 
              

            print(f'Dataset img size: {ws}')
            print(f'lc:{lc}, hc{hc}, bc{bc}}')
            print(f'config stack: {stack}, Layers:{sum(stack*2)}')
            print(f'lr:{lr}')
            print(f'ep:{n_epochs}')
            print(f1_score(y_true, y_pred))
            print(confusion_matrix(y_true, y_pred))
            print(f'====')


#check on best model different img size if time

In [ ]:
# Benchmark
# Dataset img size: 128
# config stack: (1, 1, 1, 1, 1, 1), Layers:12
# 0.5343228200371057

# Last layer
# config stack: (1, 2, 2, 2, 3, 3), Layers:26
# 0.5343228200371057
# config stack: (1, 2, 2, 3, 3, 4), Layers:30 == crash


Dataset img size: 128
lc:(4, 32, 8)
config stack: (1, 1, 1, 1, 1, 1), Layers:12
[StackConfig(n_encoder_blocks=1, n_decoder_blocks=1), StackConfig(n_encoder_blocks=1, n_decoder_blocks=1), StackConfig(n_encoder_blocks=1, n_decoder_blocks=1), StackConfig(n_encoder_blocks=1, n_decoder_blocks=1), StackConfig(n_encoder_blocks=1, n_decoder_blocks=1), StackConfig(n_encoder_blocks=1, n_decoder_blocks=1)]
0.366412213740458

In [ ]:
len( df['label'][int(((window_size)-1)):5000] ) 

In [ ]:
for k in [(1,1,1,1,1,1),(1,1,1,2,2,3),(1,1,2,2,2,3),(1,2,2,2,3,3),(1,2,2,3,3,4), (1,2,3,3,4,4), (1,2,3,4,4,5), (1,2,3,4,5,5)]: 
  print( sum(k*2) ) 

# Outtakes

In [ ]:
# w = 0.5*window_size

# # T2GAF Dataset 
# ts_windows = rolling_window(ts, window_size) #mulitple of window size 

# # Data preprocessing 
# ## Set up GAF 
# gasf = GramianAngularField(image_size=resolution, method='summation')# , sample_range=(0,1))

# ## set up list storing img of ts window
# ts_windows_img = []

# for ts_window in ts_windows:

#   ### GASF
#   X_gasf = gasf.fit_transform(ts_window.reshape(1,-1))

#   ts_windows_img.append(X_gasf[0])

# # convert to tensor
# tensor_img = torch.Tensor(ts_windows_img)

# # Data loader set up 
# X_train, X_val, _, _ = train_test_split(tensor_img, 
#                                         tensor_img,
#                                         test_size=0.1, 
#                                         random_state=42)

# ## Dataset GAF
# data_loader_train_GAF = torch.utils.data.DataLoader(preprocess_images(X_train),
#                                           batch_size=batch_size ,
#                                           shuffle=False)

# data_loader_val_GAF = torch.utils.data.DataLoader(preprocess_images(X_val),
#                                           batch_size=batch_size ,
#                                           shuffle=False)

# data_loader_test_GAF = torch.utils.data.DataLoader(preprocess_images(tensor_img),
#                                           batch_size=batch_size ,
#                                           shuffle=False)

In [ ]:
# # T2ALL
# ts_windows = rolling_window(ts, w) #mulitple of window size 
# # Data preprocessing 
# gasf = GramianAngularField(image_size=w, method='summation')
# gadf = GramianAngularField(image_size=w, method='difference')
# rp = RecurrencePlot(dimension=1, time_delay=1, threshold='point', percentage=10)
# mtf = MarkovTransitionField(image_size=w)

# ## set up list storing img of ts window
# ts_windows_img_all = []

# for ts_window in ts_windows:

#   ### GASF
#   X_gasf = gasf.fit_transform(ts_window.reshape(1,-1))

#   # ### GADF
#   X_gadf = gadf.fit_transform(ts_window.reshape(1,-1))

#   # ### RP
#   X_rp = rp.fit_transform(ts_window.reshape(1,-1))

#   # ### MTF 
#   X_mtf = mtf.fit_transform(ts_window.reshape(1,-1))

#   # ### concat/ append
#   x1 = np.concatenate((X_gadf[0], X_mtf[0]))
#   x2 = np.concatenate((X_rp[0], X_gasf[0]))

#   ts_windows_img_all.append(np.concatenate((x1,x2), 1))

# # convert to tensor
# tensor_img_all = torch.Tensor(ts_windows_img_all)

# ## Train/ Validation data
# X_train, X_val, _, _ = train_test_split(tensor_img_all, 
#                                         tensor_img_all,
#                                         test_size=0.1, 
#                                         random_state=42)

# ## Dataset all

# data_loader_train_all = torch.utils.data.DataLoader(preprocess_images(X_train),
#                                           batch_size=batch_size ,
#                                           shuffle=False)

# data_loader_val_all = torch.utils.data.DataLoader(preprocess_images(X_val),
#                                           batch_size=batch_size ,
#                                           shuffle=False)

# data_loader_val_all = torch.utils.data.DataLoader(preprocess_images(tensor_img_all),
#                                           batch_size=batch_size ,
#                                           shuffle=False)

In [ ]:
# anomaly_err_score= [] #tuple with initial pred &  r_error

# for ind, r_error in enumerate(MSEs_of_inputs): 
          
#           # Anomaly detection
#           if r_error > (mean + 2*std) or r_error < (mean - 2*std):
                
#               #print('Anomaly')
#               #df['pred'][ind+(w-1)] = 1 #w is width set initially -1 due to Xue et al implementation
#               anomaly_err_score.append(1, r_error)

#           else: 
#               #print('Normal')
#               #df['pred'][ind+(w-1)] = 0
#               anomaly_err_score.append(0, r_error)

import numpy as np 
y_pred = [0,1,1,0,0,0,0,1,1,1,1,0,0,1,1]
MSEs_of_inputs = np.random.random(15)
y_array = np.array(y_pred)
ind_seq_1 = np.where(y_array==1)[0]



In [ ]:
def sub_sequence_anomalies(predictions): 
  """ 
  Function: 
  - Filters anomaly prediction sequences

  Argument: 
  predictions = Takes list of prediction

  Output: 
  - returns ind of anomalies 
  """

  sub_seq = []
  l = []
  for ind in range(len(predictions)): 
    
    if predictions[ind]==0: 
      #normal
      if l:  #check if list is empty and if not will append
        sub_seq.append(l) # append anomaly seq
        l=[] # reset

      else: 
        continue
        
    elif predictions[ind]==1:
      #append anomaly
      l.append(ind)

      #append subsequence if it end on anomaly
      if ind == (len(predictions) - 1): 
        sub_seq.append(l)

  return sub_seq

sub_sequence_anomalies(y_pred)

In [ ]:
#filer out anomaly sequences 
sub_seq = sub_sequence_anomalies(y_pred)

#get respective maximum of each sub_seq
seq_max = []
for ss in sub_seq: 
  
  seq_max.append( np.max(MSEs_of_inputs[ss])) 

#sort in descending order 
seq_max_descent = np.sort(np.array(seq_max))[::-1]

#std of all anomalies

##std of anomalies  
ind_seq_1 = np.where(np.array(y_pred)==1)[0] #filter ind position for all anomalies
std_anomaly = np.std( MSEs_of_inputs[ind_seq_1]) 


# list of valid max values
valid_max = []

#artifically set max as first value of seqm_prev
seqm_prev =  seq_max_descent[1]

#iter 
for seqm in seq_max_descent[1:]:

  if ( ((seqm_prev - seqm )/seqm) < 0.1) and (seqm < (4*std_anomaly)) and ( seqm < (0.95*np.max(seq_max_descent)) ): #criterion Xu et al. 2021
    
    valid_max.append(seqm) 

  seqm_prev = seqm

valid_max.append(seqm) 

#correct anomalies 
y_pred_t = np.zeros(len(y_pred))

for ind, max in enumerate(seq_max): #takes index and max value

  if max in valid_max: #check if max in in valid max values 

    for k in sub_seq[ind]:  # takes sub-sequence list by index based on max value
      
      y_pred_t[k] = 1 #amend dummy y_pred

y_pred_t




In [ ]:
y_pred

In [ ]:

#sort in descending order
#desent rate of sorted maximums 𝑝𝑖 = (𝑚𝑖−1 −𝑚𝑖 )/𝑚𝑖

# if 𝑝𝑖 < 𝜃 & 𝑚𝑖 < 4 · 𝑠𝑡𝑑 &𝑚𝑖 < 𝜆 ·𝑚1 = true
# thus (𝑚𝑖−1 −𝑚𝑖 )/𝑚𝑖 < 0.1 & 𝑚𝑖 < 4 · 𝑠𝑡𝑑 & 𝑚𝑖 < 0.95 · max = true
#===> then subsequent sequences = normal 


## NOTE 
# copy setting of Xu et al. 2021
# 𝜃 = 0.1, 𝜆 = 0.95, 
# 𝑠𝑡𝑑 refers to the standard eviation of anomaly scores on the whole ts